# Preambule
#### Goal
The goal of this session is to get familiar with the Bloomberg Python API.<br> 
This will be done by building a class containing a function which mimicks the behavior of the BDP Excel function.

#### What the function will do
Our BDP-like function should be able to : <br>
1 - Retrieve reference data <br>
2 - For as many tickers as possible <br>
3 - For as many fields as possible <br>
4 - And allow for the possibility to add overrides <br>

#### References 
https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

# I. Dependencies
These are the libraries we will be using in this notebook. blpapi is the library used for Bloomberg data.

In [1]:
import blpapi
import pandas as pd
import numpy as np
import datetime as dt

# II. Set up the Bloomberg names

We here create variables using the Name class within blpapi. <br> 
This will allow to write cleaner and more concise code when refering to strings with the api.<br>
Below are only the names required for our present work. Many more exist and you can refer to the different examples within the SDK for ones of interest to your task.

In [2]:
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

# III. The BLP class
We now start to build our function within a dedicated class.<br>

A brief reminder on the class object in Python:<br>
- Classes must have a function called _\_init_\_() which is automatically executed at class initiation
- Classes can have one or several methods
- Class object need to be instaciated before using its methods

#### A. The init function

This function aims at starting the session and setting up the desired service 

#### B. The close session method:
Simply kills the session so no ghost connection remains. 

#### C. The BDP method:
3 steps: <br>
1- Create request<br>
2- Send request <br>
3- Extract data<br>




In [3]:
class BLP():
    #-----------------------------------------------------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------    
    #-----------------------------------------------------------------------------------------------------
    
    def __init__(self):
        """
            Improve this
            BLP object initialization
            Synchronus event handling
            
        """
        # Create Session object
        self.session = blpapi.Session()
        
        
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
        
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
        
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')

        print('Session open')
    
    #-----------------------------------------------------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------
    
    def bdp(self, strSecurity, strFields, strOverrideField='', strOverrideValue=''):
        
        """
            Summary:
                Reference Data Request ; Real-time if entitled, else delayed values 
                Only supports 1 override
                
                
            Input:
                strSecurity
                strFields
                strOverrideField
                strOverrideValue         
            
            Output:
               Dict 
        """
        
        #-----------------------------------------------------------------------
        # Create request
        #-----------------------------------------------------------------------
        
        # Create request
        request = self.refDataSvc.createRequest('ReferenceDataRequest')
        
        # Put field and securities in list is single field passed
        if type(strFields) == str:
            strFields = [strFields]
        
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
            
        # Append list of fields
        for strD in strFields:
            request.append('fields', strD)

        # Append list of securities
        for strS in strSecurity:
            request.append('securities', strS)

        # Add override 
        if strOverrideField != '':
            o = request.getElement('overrides').appendElement()
            o.setElement('fieldId', strOverrideField)
            o.setElement('value', strOverrideValue)

        #-----------------------------------------------------------------------
        # Send request
        #-----------------------------------------------------------------------

        requestID = self.session.sendRequest(request)
        print("Sending request")

        #-----------------------------------------------------------------------
        # Receive request                
        #-----------------------------------------------------------------------
                
        list_msg = []
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).next()
            
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break    
        
        #-----------------------------------------------------------------------
        # Extract the data 
        #-----------------------------------------------------------------------
                

        
        return 0
    
    #-----------------------------------------------------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------

    def closeSession(self):
        print("Session closed")
        self.session.stop()

# IV. Tests

In [ ]:
blp = BLP()
strFields = ["AMT_OUTSTANDING", "PX_LAST"]
date = '20200310'
test4 = blp.bdp(strSecurity=tickers, strFields = strFields, strOverrideField = "AMOUNT_OUTSTANDING_AS_OF_DT", strOverrideValue = date )
BLP.closeSession()